THIS FILE IS FOR DATA CRAWLING ONLY!!!

In [ ]:

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import numpy as np
import pandas as pd

OPEN DATA PAGE

In [ ]:
browser = webdriver.Chrome()

In [ ]:
browser.get('https://one-versus-one.com/en/rankings/all/statistics')
browser.maximize_window()
browser.implicitly_wait(20)
browser.execute_script("window.scrollTo(0, 1400)") 

ACCEPT COOKIES AND DATA PROCESSING

In [ ]:
browser.find_element(By.ID, "cmpwelcomebtnyes").click()

STORE EACH LINK TO ARRAY

In [ ]:
href = []
count = 1
while(count < 145):
    html_text = browser.page_source
    tree = BeautifulSoup(html_text)
    player_link = tree.find_all("a", "ranking-section__list-item-anchor")
    for i in player_link:
        href.append(i.get('href'))
    if count != 144:
        browser.find_element(By.XPATH, "//div[@id = 'pagination-module']/div[9]").click()
    time.sleep(1)
    count += 1 

href.pop()
href.pop(307) #dead link
href.pop(489) #dead link
href.pop(693) #dead link
href.pop(1133)  #dead link
href.pop(1144)  #dead link
href.pop(1161)  #dead link
href.pop(1261)  #dead link


In [ ]:
href.pop(1261)  #dead link

In [ ]:
print(href[1261])

CRAWL DATA FROM EACH PLAYER

In [65]:
player_data = []
for i in range(len(href)):
    browser.get(href[i])
    html_text = browser.page_source
    tree = BeautifulSoup(html_text)
    player_name = tree.find_all("h1", "players-page-header__name section-big-text")
    no_data = tree.find_all("div", "player-no-data-warning")
    if (no_data != []):
        continue
    for i in range(len(player_name)):
        player_name[i] = player_name[i].get_text()
    record_tab = tree.find_all("div", "records__total-switch records-total-switch")
    hl = tree.find_all("td", "scouting-report__item-metric")
    if (record_tab != [] and hl != []):
        browser.execute_script("window.scrollTo(0, 1650)") 
        time.sleep(0.5)
        browser.execute_script("window.scrollTo(0, 1675)")
    elif (record_tab != [] and hl == []):
        browser.execute_script("window.scrollTo(0, 1400)") 
        time.sleep(0.5)
        browser.execute_script("window.scrollTo(0, 1400)")
    elif(record_tab == []):
        browser.execute_script("window.scrollTo(0, 1200)") 
        time.sleep(0.5)
        browser.execute_script("window.scrollTo(0, 1200)")
    time.sleep(0.1)
    browser.find_element(By.XPATH, "//div[@class='btn btn-outline-light scouting-report__button']").click()
    player_stats = []
    while (len(player_stats) <= 5):
        html_text = browser.page_source
        tree = BeautifulSoup(html_text)
        player_stats = tree.find_all("td", "scouting-report__item-ninety")
    if (len(player_stats) == 33):
        player_stats.pop(0)
        player_stats.pop(0)
        player_stats.pop(0)
        player_stats.pop(0)
        player_stats.pop(0)
    for j in range(len(player_stats)):
        player_stats[j] = float(player_stats[j].get_text())
    player_data.append(player_stats)

TRANSPOSE THE DATA

In [69]:
player_data_X = [[player_data[j][i] for j in range(len(player_data))] for i in range(len(player_data[0]))]
player_data_X = np.array(player_data_X)


INPUT DATA TO DATAFRAME

In [70]:
data = {"CHALLENGES WON, %": player_data_X[0],
        "AIR CHALLENGES LOST": player_data_X[1],
        "AIR CHALLENGES WON": player_data_X[2],
        "FOULS": player_data_X[3],
        "YELLOW CARDS": player_data_X[4],
        "AIR CHALLENGES, %": player_data_X[5],
        "ASSISTS": player_data_X[6],
        "EXPECTED ASSISTS (XA)": player_data_X[7],
        "PASSES INTO FINAL THIRD": player_data_X[8],
        "CROSSES": player_data_X[9],
        "SUCCESSFUL DRIBBLES, %": player_data_X[10],
        "TACKLES": player_data_X[11],
        "PASS INTERCEPTIONS": player_data_X[12],
        "ERRORS LEADING TO GOAL": player_data_X[13],
        "BLOCKED SHOTS": player_data_X[14],
        "CLEARANCES": player_data_X[15],
        "TACKLES WON, %": player_data_X[16],
        "PASSES": player_data_X[17],
        "PASS COMPLETION, %": player_data_X[18],
        "PASSING SPEED": player_data_X[19],
        "GOALS": player_data_X[20],
        "BIG CHANCES": player_data_X[21],
        "CONVERSION OF BIG CHANCES, %": player_data_X[22],
        "SHOTS ON TARGET": player_data_X[23],
        "SHOTS ON TARGET CONVERSION RATE, %": player_data_X[24],
        "EXPECTED GOALS (XG)": player_data_X[25],
        "GOALS WITH HEAD": player_data_X[26],
        "XG CONVERSION": player_data_X[27],
        }   
df = pd.DataFrame.from_dict(data)


EXPORT TO CSV FILE

In [71]:
df.to_csv("data4.csv")